In [1]:
!pip install langchain
!pip install langchain_community
!pip install langchain_experimental
!pip install openai
!pip install docarray
!pip install tiktoken
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras

In [2]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool, PythonREPL

from openai import OpenAI
import openai
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api")
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

llm_model = "gpt-3.5-turbo"

In [3]:
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.9, model=llm_model, openai_api_key=openai.api_key)


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

math agent

In [5]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [6]:
agent("If you have a rectangular garden with dimensions 12 meters by 8 meters, what is the total area of the garden in square meters?")


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: We can calculate the total area of the garden by multiplying the length and width of the rectangle.
Action:
```
{
  "action": "Calculator",
  "action_input": "12 * 8"
}
```
Observation: Answer: 96
Thought:I now know the final answer
Final Answer: The total area of the garden is 96 square meters.

> Finished chain.


{'input': 'If you have a rectangular garden with dimensions 12 meters by 8 meters, what is the total area of the garden in square meters?',
 'output': 'The total area of the garden is 96 square meters.'}

wikipedia agent

In [7]:
question = "Andrew Ng is a British-American computer scientist \
and the Founder of Coursera and Deep Learning.ai and a Professor at Stanford University\
what books did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: We can use Wikipedia to find out which books Andrew Ng has written.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Andrew Ng"
}
```
Observation: Page: Andrew Ng
Summary: Andrew Yan-Tak Ng (Chinese: 吳恩達; born 1976) is a British-American computer scientist and technology entrepreneur focusing on machine learning and artificial intelligence (AI). Ng was a cofounder and head of Google Brain and was the former Chief Scientist at Baidu, building the company's Artificial Intelligence Group into a team of several thousand people.
Ng is an adjunct professor at Stanford University (formerly associate professor and Director of its Stanford AI Lab or SAIL). Ng has also worked in the field of online education, cofounding Coursera and DeepLearning.AI. He has spearheaded many efforts to "democratize deep learning" teaching over 8 million students through his online courses. Ng is renowned globally in computer science, recognized in Time mag

In [8]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [9]:
employee_list = [["Smith", "John", 35], 
                 ["Doe", "Jane", 28],
                 ["Black", "Michael", 42],
                 ["Brown", "Emily", 31], 
                 ["White", "David", 39], 
                 ["Green", "Sarah", 45],
                 ["Jones", "Christopher", 37]
                ]

In [10]:
agent.run(f""" Sort these employees by their age in ascending order and \
then by last name in descending order, and print the output: {employee_list}""")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
We need to sort the list of employees by age in ascending order first, then by last name in descending order.
Action: Python_REPL
Action Input: employees = [['Smith', 'John', 35], ['Doe', 'Jane', 28], ['Black', 'Michael', 42], ['Brown', 'Emily', 31], ['White', 'David', 39], ['Green', 'Sarah', 45], ['Jones', 'Christopher', 37]]
employees.sort(key=lambda x: (x[2], x[0]), reverse=False)
print(employees)
Observation: [['Doe', 'Jane', 28], ['Brown', 'Emily', 31], ['Smith', 'John', 35], ['Jones', 'Christopher', 37], ['White', 'David', 39], ['Black', 'Michael', 42], ['Green', 'Sarah', 45]]

Thought:Now we need to sort the list by last name in descending order.
Action: Python_REPL
Action Input: employees.sort(key=lambda x: x[0], reverse=True)
print(employees)
Observation: [['White', 'David', 39], ['Smith', 'John', 35], ['Jones', 'Christopher', 37], ['Green', 'Sarah', 45], ['Doe', 'Jane', 28], ['Brown', 'Emily', 31], ['Black', 'Michael', 42]]

Thought:I n

"[['White', 'David', 39], ['Smith', 'John', 35], ['Jones', 'Christopher', 37], ['Green', 'Sarah', 45], ['Doe', 'Jane', 28], ['Brown', 'Emily', 31], ['Black', 'Michael', 42]]"

In [11]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {employee_list}""") 
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Smith', 'John', 35], ['Doe', 'Jane', 28], ['Black', 'Michael', 42], ['Brown', 'Emily', 31], ['White', 'David', 39], ['Green', 'Sarah', 45], ['Jones', 'Christopher', 37]]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Smith', 'John', 35], ['Doe', 'Jane', 28], ['Black', 'Michael', 42], ['Brown', 'Emily', 31], ['White', 'David', 39], ['Green', 'Sarah', 45], ['Jones', 'Christopher', 37]]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a p

customized agent

In [12]:
from langchain.agents import tool
from datetime import date

In [13]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [14]:
agent = initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    handle_parsing_errors=True, 
    verbose=True
)


In [15]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I should use the `time` tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
``` 

Observation: 2024-06-16
Thought:Final Answer: 2024-06-16

> Finished chain.
